In [2]:
from datetime import datetime, timedelta
import pandas as pd
import requests, json
from pandas import json_normalize

In [3]:
today = datetime.now()

In [4]:
today.strftime('%Y%m%y')

tyear = today.year
tmon = today.month
tday = today.day

# tyear = 2022
# tmon = 12
# 해당 날짜를 주면 datetime으로 형식을 변환해 준다.
c_date = f'{tyear:02d}-{tmon:02d}-{tday:02d}'
ser_day = datetime.strptime(c_date, '%Y-%m-%d')


In [5]:
import calendar

# 0이면 월요일, 6이면 일요일이 시작일이다.
start_day_week, end_day = calendar.monthrange(tyear, tmon)
print(start_day_week, end_day)

# 해당 달의 첫날과 마지막날 datetime형식으로 변환한다.
start_date = datetime.strptime(f'{tyear:02d}-{tmon:02d}-01', '%Y-%m-%d')
end_date = datetime.strptime(f'{tyear:02d}-{tmon:02d}-{end_day:02d}', '%Y-%m-%d')
print(start_date, end_date)


0 31
2023-05-01 00:00:00 2023-05-31 00:00:00


In [6]:
# 시작날짜를 주고 periods만큰 날을 계산한다.
[nd.strftime('%Y%m%d') for nd in pd.date_range(start_date.strftime('%Y-%m-%d'), periods=3)]


['20230501', '20230502', '20230503']

In [7]:
# 현재 날짜에서 전 달 구하기
lines, style = [], []
l_dict = dict()

before_month_day = 0
if start_day_week < 6:
    before_month_day = (start_date - timedelta(start_day_week+1))
    print(before_month_day)

    lines.extend([ nd.strftime('%Y%m%d') for nd in pd.date_range(before_month_day.strftime('%Y-%m-%d'), periods=start_day_week+1)])
    style = ['0.5' for i in range(start_day_week+1)]  # opacity 0.5
    # for i in range(start_day_week+1):
    #     style.append('0.5')
        
print(lines)
print(style)
    


2023-04-30 00:00:00
['20230430']
['0.5']


In [8]:
# 현재 달
lines.extend([nd.strftime('%Y%m%d') for nd in pd.date_range(start_date.strftime('%Y-%m-%d'), periods=end_day)])
style.extend(['' for i in range(end_day)])

In [9]:
# lines.extend(range(1, end_day+1))
print(lines)
print(len(style))

['20230430', '20230501', '20230502', '20230503', '20230504', '20230505', '20230506', '20230507', '20230508', '20230509', '20230510', '20230511', '20230512', '20230513', '20230514', '20230515', '20230516', '20230517', '20230518', '20230519', '20230520', '20230521', '20230522', '20230523', '20230524', '20230525', '20230526', '20230527', '20230528', '20230529', '20230530', '20230531']
32


In [10]:
# 빈 칸에 다음 달 날짜 구하기
nday = calendar.weekday(tyear, tmon, end_day)
next_cnt = 0
if nday == 6: next_cnt = 6
elif nday < 5: next_cnt = 5 - nday

# 마지막 날짜 구하기
lines.extend([nd.strftime('%Y%m%d') for nd in pd.date_range((end_date+timedelta(1)).strftime('%Y-%m-%d'), periods=next_cnt)])
style.extend(['0.5' for i in range(next_cnt)])

# lines.extend(range(1, next_cnt+1))
# print(lines)

In [11]:
l_dt = pd.DataFrame({'locdate':lines, 'opacity': style})
# l_dt['day'] = l_dt.locdate.apply(lambda x: int(x.strftime('%d')))
l_dt.head(7)

,locdate,opacity
0,20230430,0.5
1,20230501,
2,20230502,
3,20230503,
4,20230504,
5,20230505,
6,20230506,


In [12]:
for n in lines[:5]:
    print(n)

20230430
20230501
20230502
20230503
20230504


In [13]:
with open('../static/key/holidays.txt') as f:
    apikey = f.read()

- 공휴일

In [14]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey

In [15]:
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text


print(res)

{"response":{"header":{"resultCode":"00","resultMsg":"NORMAL SERVICE."},"body":{"items":{"item":[{"dateKind":"01","dateName":"1월1일","isHoliday":"Y","locdate":20230101,"seq":1},{"dateKind":"01","dateName":"설날","isHoliday":"Y","locdate":20230121,"seq":1},{"dateKind":"01","dateName":"설날","isHoliday":"Y","locdate":20230122,"seq":1},{"dateKind":"01","dateName":"설날","isHoliday":"Y","locdate":20230123,"seq":1},{"dateKind":"01","dateName":"대체공휴일","isHoliday":"Y","locdate":20230124,"seq":1},{"dateKind":"01","dateName":"삼일절","isHoliday":"Y","locdate":20230301,"seq":1},{"dateKind":"01","dateName":"어린이날","isHoliday":"Y","locdate":20230505,"seq":1},{"dateKind":"01","dateName":"부처님오신날","isHoliday":"Y","locdate":20230527,"seq":1},{"dateKind":"01","dateName":"대체공휴일","isHoliday":"Y","locdate":20230529,"seq":1},{"dateKind":"01","dateName":"현충일","isHoliday":"Y","locdate":20230606,"seq":1},{"dateKind":"01","dateName":"광복절","isHoliday":"Y","locdate":20230815,"seq":1},{"dateKind":"01","dateName":"추석","isHol

In [16]:
res = json.loads(res)
res.keys()

dict_keys(['response'])

In [17]:
res['response']['body']['items']['item']

[{'dateKind': '01',
  'dateName': '1월1일',
  'isHoliday': 'Y',
  'locdate': 20230101,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '설날',
  'isHoliday': 'Y',
  'locdate': 20230121,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '설날',
  'isHoliday': 'Y',
  'locdate': 20230122,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '설날',
  'isHoliday': 'Y',
  'locdate': 20230123,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '대체공휴일',
  'isHoliday': 'Y',
  'locdate': 20230124,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '삼일절',
  'isHoliday': 'Y',
  'locdate': 20230301,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '어린이날',
  'isHoliday': 'Y',
  'locdate': 20230505,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '부처님오신날',
  'isHoliday': 'Y',
  'locdate': 20230527,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '대체공휴일',
  'isHoliday': 'Y',
  'locdate': 20230529,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '현충일',
  'isHoliday': 'Y',
  'locdate': 20230606,
  'seq': 1},
 {'dateKind': '01',
  'd

In [18]:
holidays = json_normalize(res['response']['body']['items']['item'])
holidays

,dateKind,dateName,isHoliday,locdate,seq
0,01,1월1일,Y,20230101,1
1,01,설날,Y,20230121,1
2,01,설날,Y,20230122,1
3,01,설날,Y,20230123,1
4,01,대체공휴일,Y,20230124,1
5,01,삼일절,Y,20230301,1
6,01,어린이날,Y,20230505,1
7,01,부처님오신날,Y,20230527,1
8,01,대체공휴일,Y,20230529,1
9,01,현충일,Y,20230606,1


- 24절기

In [19]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/get24DivisionsInfo?_type=json&numOfRows=50&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

res = json.loads(res)
res.keys()

dict_keys(['response'])

In [20]:
json_normalize(res['response']['body']['items']['item'])[['dateKind', 'dateName', 'isHoliday', 'locdate']]

,dateKind,dateName,isHoliday,locdate
0,03,소한,N,20230106
1,03,대한,N,20230120
2,03,입춘,N,20230204
3,03,우수,N,20230219
4,03,경칩,N,20230306
5,03,춘분,N,20230321
6,03,청명,N,20230405
7,03,곡우,N,20230420
8,03,입하,N,20230506
9,03,소만,N,20230521


- 기념일

In [21]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getAnniversaryInfo?_type=json&numOfRows=80&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

res = json.loads(res)
dt = pd.concat([holidays, json_normalize(res['response']['body']['items']['item'])])


In [22]:
res['response']['body']['items']['item']

[{'dateKind': '02',
  'dateName': '2·28 민주운동 기념일',
  'isHoliday': 'N',
  'locdate': 20230228,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '납세자의 날',
  'isHoliday': 'N',
  'locdate': 20230303,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '3·8민주의거 기념일',
  'isHoliday': 'N',
  'locdate': 20230308,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '3·15 의거 기념일',
  'isHoliday': 'N',
  'locdate': 20230315,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '상공의 날',
  'isHoliday': 'N',
  'locdate': 20230315,
  'seq': 2},
 {'dateKind': '02',
  'dateName': '서해수호의 날',
  'isHoliday': 'N',
  'locdate': 20230324,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '4·3희생자 추념일',
  'isHoliday': 'N',
  'locdate': 20230403,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '식목일',
  'isHoliday': 'N',
  'locdate': 20230405,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '예비군의 날',
  'isHoliday': 'N',
  'locdate': 20230407,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '보건의 날',
  'isHoliday': 'N',
  'locdate': 202304

- 잡절


In [23]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getSundryDayInfo?_type=json&numOfRows=50&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

res = json.loads(res)
json_normalize(res['response']['body']['items']['item'])

,dateKind,dateName,isHoliday,locdate,seq
0,04,정월대보름,N,20230205,1
1,04,한식,N,20230406,1
2,04,단오,N,20230622,1
3,04,초복,N,20230711,2
4,04,중복,N,20230721,1
5,04,말복,N,20230810,1
6,04,칠석,N,20230822,1


- 한 번에 구하기

In [24]:
def is_info_day(in_list, s_date, s_datename):
    # print(type(lists))
    for i in range(len(in_list)):
        # print( in_list[i]['locdate'], s_date)
        if in_list[i]['locdate'] == s_date:
            in_list[i]['dateName'] = in_list[i]['dateName'] + ' ' + s_datename
            # print(in_list[i]['dateName'])
            return True
    return False

In [47]:

with open('../static/key/holidays.txt') as f:
    apikey = f.read()
    
f_list = ['getRestDeInfo', 'getAnniversaryInfo', 'get24DivisionsInfo', 'getSundryDayInfo']
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/' #getSundryDayInfo?_type=json&numOfRows=50&solYear='
param1 = '?_type=json&numOfRows=80&solYear='

holidays = None
in_list = []

for name in f_list:
    url = base_url + name + param1 + str(tyear) + '&ServiceKey=' + apikey
    ress = ''

    try:
        result = requests.get(url) 
        if result.status_code == 200:
            ress = result.text
    except:
        print('CODE = ', result.status_code)
        pass

    if ress: 
        ress = json.loads(ress)['response']['body']['items']['item']
        for i in range(len(ress)):
            # print(ress[i]['locdate'])
            if is_info_day(in_list, ress[i]['locdate'], ress[i]['dateName']) :
                pass
            else:
                in_list.append(ress[i])


holidays = pd.DataFrame(in_list)
holidays = holidays[['locdate', 'dateName', 'isHoliday']]




In [40]:
holidays.head()

,locdate,dateName,isHoliday
0,20230101,1월1일,Y
1,20230121,설날,Y
2,20230122,설날,Y
3,20230123,설날,Y
4,20230124,대체공휴일,Y


In [27]:
holidays.isna().sum().sum()

82

In [41]:
holidays['locdate'] = holidays['locdate'].astype(str)

In [42]:
holidays

,locdate,dateName,isHoliday
0,20230101,1월1일,Y
1,20230121,설날,Y
2,20230122,설날,Y
3,20230123,설날,Y
4,20230124,대체공휴일,Y
...,...,...,...
94,20230406,한식,N
95,20230622,단오,N
96,20230721,중복,N
97,20230810,말복,N


In [50]:
try:
    nindex = holidays.index[holidays.dateName=='기독탄신일'][0]
    holidays.iloc[nindex, 1] = '크리스마스'
    nindex = holidays.index[holidays.dateName=='1월1일'][0]
    holidays.iloc[nindex, 1] = '신정'
except:
    pass



In [51]:
holidays[(holidays.dateName=='크리스마스') | (holidays.dateName=='신정')]

,locdate,dateName,isHoliday
0,20230101,신정,Y
16,20231225,크리스마스,Y


In [57]:
holidays.to_csv(f'../static/data/holidays{tyear}.csv', index=False)


In [58]:
try:
    holidays = pd.read_csv(f'../static/data/holidays{tyear}.csv')
    holidays
except:
    pass

In [61]:
len(holidays)


99

In [46]:
result_dt = pd.merge(l_dt, holidays, how='left', on='locdate')
# result_dt.dateKind.fillna('05', inplace=True)
result_dt.dateName.fillna('', inplace=True)
result_dt.isHoliday.fillna('N', inplace=True)
result_dt = result_dt[['locdate', 'opacity', 'dateName', 'isHoliday']]
# result_dt.drop(labels=['seq', 'kst', 'sunLongitude'], axis=1, inplace=True)
result_dt

,locdate,opacity,dateName,isHoliday
0,20230430,0.5,,N
1,20230501,,근로자의 날,N
2,20230502,,,N
3,20230503,,,N
4,20230504,,,N
5,20230505,,어린이날,Y
6,20230506,,입하,N
7,20230507,,,N
8,20230508,,어버이 날,N
9,20230509,,,N


In [34]:
res_list = result_dt.to_dict('records')


In [35]:
res_list


[{'locdate': '20230430', 'opacity': '0.5', 'dateName': '', 'isHoliday': 'N'},
 {'locdate': '20230501',
  'opacity': '',
  'dateName': '근로자의 날',
  'isHoliday': 'N'},
 {'locdate': '20230502', 'opacity': '', 'dateName': '', 'isHoliday': 'N'},
 {'locdate': '20230503', 'opacity': '', 'dateName': '', 'isHoliday': 'N'},
 {'locdate': '20230504', 'opacity': '', 'dateName': '', 'isHoliday': 'N'},
 {'locdate': '20230505', 'opacity': '', 'dateName': '어린이날', 'isHoliday': 'Y'},
 {'locdate': '20230506', 'opacity': '', 'dateName': '입하', 'isHoliday': 'N'},
 {'locdate': '20230507', 'opacity': '', 'dateName': '', 'isHoliday': 'N'},
 {'locdate': '20230508', 'opacity': '', 'dateName': '어버이 날', 'isHoliday': 'N'},
 {'locdate': '20230509', 'opacity': '', 'dateName': '', 'isHoliday': 'N'},
 {'locdate': '20230510',
  'opacity': '',
  'dateName': '유권자의 날',
  'isHoliday': 'N'},
 {'locdate': '20230511',
  'opacity': '',
  'dateName': '동학농민혁명 기념일',
  'isHoliday': 'N'},
 {'locdate': '20230512', 'opacity': '', 'dateN